In [1]:
import requests
import warnings
import pandas as pd
from bs4 import BeautifulSoup
warnings.filterwarnings('ignore')

In [2]:
url = 'https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-qualifier-a-2021-22-1299556/ireland-vs-united-arab-emirates-final-1299585/live-cricket-score'
url = url.split('/')
url = '/'.join(url[: -1]) + '/'

In [4]:
response = requests.get(url + 'live-cricket-score')
soup = BeautifulSoup(response.text, 'html.parser')

#teams

teams_1 = soup.find_all('div', attrs = {'class' : 'match-header'})
for team in teams_1:
    teams_2 = team.find_all('p', attrs = {'class' : 'name'})
teams = [team.text for team in teams_2]

#innings

match_status = soup.find_all('div', attrs = {'class' : 'match-info match-info-MATCH match-info-MATCH-full-width'})
for ms in match_status:
    verifier = ms.find('div', attrs = {'class' : 'status-text'})
if 'chose to' in verifier.text:
    innings = 1
elif 'need' in verifier.text:
    innings = 2
else:
    innings = 0

response = requests.get(url + 'full-scorecard')
soup = BeautifulSoup(response.text, 'html.parser')

#venue

venues_1 = soup.find('div', attrs = {'class' : 'match-scorecard-page'})
for ven in venues_1:
    venues_2 = ven.find_all('a')
venue = venues_2[0].text

#toss

toss_result = soup.find_all('tr')
toss = [t for t in toss_result if 'Toss' in t.text]
toss_winner = toss[0].text.split(',')[0].split('Toss')[1]
toss_loser = [team for team in teams if team != toss_winner]
toss_loser = toss_loser[0]

#batting and bowling team

if 'bat' in toss[0].text and innings == 1:
    batting_team = toss_winner
    bowling_team = toss_loser
elif 'bat' in toss[0].text and innings == 2:
    batting_team = toss_loser
    bowling_team = toss_winner
elif 'field' in toss[0].text and innings == 1:
    batting_team = toss_loser
    bowling_team = toss_winner
else:
    batting_team = toss_winner
    bowling_team = toss_loser

#over, score and wickets

status = soup.find_all('tfoot')
if innings == 1:
    overs = float(status[0].text.split(' Ov')[0].split('(')[1])
    score = int(status[0].text.split('/')[0].split(')')[-1])
    if '/' in status[0].text:
        wickets = int(status[0].text.split('/')[1][0])
    else:
        wickets = 10
else:
    overs = float(status[1].text.split(' Ov')[0].split('(')[1])
    score = int(status[1].text.split('/')[0].split(')')[-1])
    if '/' in status[1].text:
        wickets = int(status[1].text.split('/')[1][0])
    else:
        wickets = 10

#last five overs

if overs >= 5.1:
    recent = soup.find('div', attrs = {'class' : 'recent-overs'})
    last_5_score = int(recent.text.split('/')[0].split('\xa0')[-1])
    last_5_wickets = int(recent.text.split('/')[-1].split(' ')[0])
else:
    pass

In [5]:
teams

['Ireland', 'United Arab Emirates']

In [6]:
venue

'Al Amerat Cricket Ground Oman Cricket (Ministry Turf 1)'

In [7]:
toss_winner

'Ireland'

In [8]:
innings

2

In [9]:
batting_team

'United Arab Emirates'

In [10]:
bowling_team

'Ireland'

In [11]:
overs

12.0

In [12]:
score

86

In [13]:
wickets

2

In [14]:
last_5_score

51

In [15]:
last_5_wickets

0